In [8]:
# 필요한 모듈 임포트
import tensorflow as tf
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# 데이터 경로 설정
train_dir = 'car-damage-dataset/data1a/training'  # 학습 데이터 폴더 경로
validation_dir = 'car-damage-dataset/data1a/validation'  # 검증 데이터 폴더 경로

# ImageDataGenerator 설정
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

validation_datagen = ImageDataGenerator(rescale=1./255)

# ImageDataGenerator에서 이미지 불러오기
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(256, 256),  # 이미지 크기 256x256로 조정
    batch_size=32,
    class_mode='binary'  # 이진 분류
)

validation_generator = validation_datagen.flow_from_directory(
    validation_dir,
    target_size=(256, 256),  # 이미지 크기 256x256로 조정
    batch_size=32,
    class_mode='binary'
)

# 모델 설계
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(256, 256, 3)),
    MaxPooling2D(pool_size=(2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D(pool_size=(2, 2)),
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D(pool_size=(2, 2)),
    Flatten(),
    Dense(512, activation='relu'),  # 은닉층 크기 설정
    Dense(1, activation='sigmoid')  # 이진 분류를 위한 출력층
])

# 모델 컴파일
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# 모델 학습
history = model.fit(
    train_generator,
    steps_per_epoch=len(train_generator),
    epochs=200,  # 에포크 수 설정
    validation_data=validation_generator,
    validation_steps=len(validation_generator)
)


Found 3819 images belonging to 2 classes.
Found 801 images belonging to 2 classes.


c:\Users\kangdongwoo\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/200
120/120 ━━━━━━━━━━━━━━━━━━━━ 108s 863ms/step - accuracy: 0.7381 - loss: 0.8993 - val_accuracy: 0.7041 - val_loss: 0.5512
Epoch 2/200
120/120 ━━━━━━━━━━━━━━━━━━━━ 0s 292us/step - accuracy: 0.0000e+00 - loss: 0.0000e+00
Epoch 3/200
120/120 ━━━━━━━━━━━━━━━━━━━━ 103s 835ms/step - accuracy: 0.7657 - loss: 0.5095 - val_accuracy: 0.7678 - val_loss: 0.5130
Epoch 4/200
120/120 ━━━━━━━━━━━━━━━━━━━━ 0s 141us/step - accuracy: 0.0000e+00 - loss: 0.0000e+00
Epoch 5/200
120/120 ━━━━━━━━━━━━━━━━━━━━ 105s 854ms/step - accuracy: 0.7852 - loss: 0.4909 - val_accuracy: 0.7978 - val_loss: 0.4806
Epoch 6/200
120/120 ━━━━━━━━━━━━━━━━━━━━ 0s 168us/step - accuracy: 0.0000e+00 - loss: 0.0000e+00
Epoch 7/200
120/120 ━━━━━━━━━━━━━━━━━━━━ 104s 847ms/step - accuracy: 0.7821 - loss: 0.4865 - val_accuracy: 0.8177 - val_loss: 0.4246
Epoch 8/200
120/120 ━━━━━━━━━━━━━━━━━━━━ 0s 172us/step - accuracy: 0.0000e+00 - loss: 0.0000e+00
Epoch 9/200
120/120 ━━━━━━━━━━━━━━━━━━━━ 104s 842ms/step - accuracy: 0.8011 - lo

In [9]:
# 모델 저장
model.save('car_damage_detection_model.h5')


In [5]:
import os
import numpy as np
from tensorflow.keras.preprocessing import image
import tensorflow as tf

# 모델 불러오기
model = tf.keras.models.load_model('../car_damage_detection_model.h5')

# 예측할 이미지가 있는 폴더 경로
img_dir = '../test_image'  # 예측할 이미지 폴더 경로

# 이미지 파일 확장자 목록
valid_extensions = ('.jpg', '.jpeg', '.png', '.bmp')

# 폴더 내 모든 이미지 파일에 대해 반복하며 예측
for img_file in os.listdir(img_dir):
    if img_file.lower().endswith(valid_extensions):  # 이미지 파일만 선택
        img_path = os.path.join(img_dir, img_file)
        try:
            # 이미지 불러오기 및 전처리
            img = image.load_img(img_path, target_size=(256, 256))  # 모델의 입력 크기에 맞게 조정
            img_array = image.img_to_array(img) / 255.0  # 정규화 (0~1 범위)
            img_array = np.expand_dims(img_array, axis=0)  # 배치 차원 추가 (1, 128, 128, 3)

            # 모델 예측
            prediction = model.predict(img_array)

            predicted_prob = prediction[0][0]   
            # 예측 결과 출력 (이진 분류)
            if predicted_prob > 0.5:
                print(f"{img_file}: 비파손 (Damage-free)")
            else:
                print(f"{img_file}: 파손 (Damaged)")

        except Exception as e:
            print(f"Error processing {img_file}: {e}")


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 74ms/step
KakaoTalk_20241114_153120613.jpg: 비파손 (Damage-free)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
KakaoTalk_20241114_153120613_01.jpg: 파손 (Damaged)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
KakaoTalk_20241114_153120613_02.jpg: 비파손 (Damage-free)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
KakaoTalk_20241114_153120613_03.jpg: 비파손 (Damage-free)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
KakaoTalk_20241114_153120613_04.jpg: 비파손 (Damage-free)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
KakaoTalk_20241114_153120613_05.jpg: 비파손 (Damage-free)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
KakaoTalk_20241114_153120613_06.jpg: 비파손 (Damage-free)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
KakaoTalk_20241114_153120613_07.jpg: 파손 (Damaged)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
KakaoTalk_20241114_153120613_08.jpg: 비파손 (Damage-free)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
KakaoTalk_20241114_153120613_09.jpg: 비파손 (Damage-free)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
KakaoTalk_20241114_153120613_10.jpg: 파손 (Dama

In [2]:

import os
import numpy as np
from tensorflow.keras.preprocessing import image
import tensorflow as tf
from flask import Flask, request, jsonify

# 모델 불러오기
model = tf.keras.models.load_model('../car_damage_detection_model.h5')

# 이미지 파일 확장자 목록
valid_extensions = ('.jpg', '.jpeg', '.png', '.bmp')

# 폴더 내 모든 이미지 파일에 대해 예측하는 함수
def predict_folder(img_dir):
    results = []
    for img_file in os.listdir(img_dir):
        if img_file.lower().endswith(valid_extensions):  # 이미지 파일만 선택
            img_path = os.path.join(img_dir, img_file)
            try:
                # 이미지 불러오기 및 전처리
                img = image.load_img(img_path, target_size=(256, 256))  # 모델의 입력 크기에 맞게 조정
                img_array = image.img_to_array(img) / 255.0  # 정규화 (0~1 범위)
                img_array = np.expand_dims(img_array, axis=0)  # 배치 차원 추가 (1, 128, 128, 3)

                # 모델 예측
                prediction = model.predict(img_array)
                predicted_prob = prediction[0][0]

                # 예측 결과 저장
                if predicted_prob > 0.5:
                    results.append({'filename': img_file, 'result': "파손되지 않은 차량입니다."})
                else:
                    results.append({'filename': img_file, 'result': "파손된 차량입니다."})
            except Exception as e:
                results.append({'filename': img_file, 'error': str(e)})
    return results

# Flask 앱 생성
app = Flask(__name__)

# 예측을 위한 엔드포인트 정의
@app.route('/predict_folder', methods=['POST'])
def predict_folder_endpoint():
    data = request.get_json()
    img_dir = data.get('img_dir')

    if img_dir and os.path.isdir(img_dir):
        results = predict_folder(img_dir)
        return jsonify({'results': results})
    else:
        return jsonify({'error': 'Invalid directory path provided.'}), 400

if __name__ == '__main__':
    app.run(debug=True)


 * Serving Flask app '__main__'
 * Debug mode: on


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug: * Restarting with watchdog (windowsapi)


SystemExit: 1

C:\Users\kangdongwoo\AppData\Roaming\Python\Python310\site-packages\IPython\core\interactiveshell.py:3386: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
